In [57]:
import os
# os.environ["MKL_NUM_THREADS"] = "1" 
# os.environ["NUMEXPR_NUM_THREADS"] = "1" 
# os.environ["OMP_NUM_THREADS"] = "1" 

import numpy as np
np.random.seed(1)
import random
random.seed(1)
import pandas as pd
import cv2
import timeit
from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
from skimage.morphology import square, dilation, watershed, erosion
from skimage import io

from shapely.wkt import loads
from shapely.geometry import mapping, Polygon, MultiPolygon


# import matplotlib.pyplot as plt
# import seaborn as sns

import json


# def mask_for_polygon(poly, im_size=(1024, 1024)):
#     img_mask = np.zeros(im_size, np.uint8)
#     int_coords = lambda x: np.array(x).round().astype(np.int32)
#     if(poly.geom_type=='MultiPolygon'):
#         exteriors = [list(x.exterior.coords) for x in poly.geoms]
#         interiors = []
#         for p in poly.geoms:
#             interiors.extend([int_coords(pi.coords) for pi in p.interiors if pi.coords is not None])
        
#     else:
#         exteriors = [int_coords(poly.exterior.coords)]
#         interiors = [int_coords(pi.coords) for pi in poly.interiors]
    
#     cv2.fillPoly(img_mask, exteriors, 1)
#     cv2.fillPoly(img_mask, interiors, 0)
#     return img_mask

def mask_for_polygon(geom, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    int_coords = lambda x: np.array(x).round().astype(np.int32)

    # If the geometry is a Polygon
    if isinstance(geom, Polygon):
        exteriors = [int_coords(geom.exterior.coords)]
        interiors = [int_coords(pi.coords) for pi in geom.interiors]
        cv2.fillPoly(img_mask, exteriors, 1)
        cv2.fillPoly(img_mask, interiors, 0)

    # If the geometry is a MultiPolygon
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            exteriors = [int_coords(poly.exterior.coords)]
            interiors = [int_coords(pi.coords) for pi in poly.interiors]
            cv2.fillPoly(img_mask, exteriors, 1)
            cv2.fillPoly(img_mask, interiors, 0)

    # If the geometry is a Point
    elif isinstance(geom, Point):
        coords = int_coords(geom.coords[:1])  # Get the first coordinate tuple
        try:
            img_mask[coords[0][1], coords[0][0]] = 1  # y, x order
        except:
            None

    return img_mask



damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 1,
    "No visible damage": 1,
    "Possibly damaged": 2,
    "Damaged":3,
    "Destroyed":4
}


def process_image(json_file):
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    cv2.imwrite(json_file.replace('_wkt_', '_mask_').replace('.json', '.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(json_file.replace('_wkt_', '_mask_').replace('_pre_disaster', '_post_disaster').replace('.json', '.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    
    
def process_image_updated(json_file):
    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    if('pre_disaster' in json_file):
        js1 = json.load(open(json_file))

        for feat in js1['features']['xy']:
            poly = loads(feat['wkt'])
            _msk = mask_for_polygon(poly)
            msk[_msk > 0] = 255

        cv2.imwrite(json_file.replace('_wkt_', '_mask_').replace('.json', '.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])

    if('post_disaster' in json_file):
        js2 = json.load(open(json_file))
        for feat in js2['features']['xy']:
            poly = loads(feat['wkt'])
            subtype = feat['properties']['subtype']
            _msk = mask_for_polygon(poly)
            msk_damage[_msk > 0] = damage_dict[subtype]
        cv2.imwrite(json_file.replace('_wkt_', '_mask_').replace('.json', '.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])

In [60]:
def list_json_files_recursive(directory):
    json_file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_file_list.append(os.path.join(root, file))
    return json_file_list

def list_subdirectories(directory_path):
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return subdirectories


label_dirs = 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23/'
list_json_files_recursive(label_dirs)[:5]

['turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23/Osmaniye/Osmaniye_20221227_10300100DF069700_pre_disaster_2_15.json',
 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23/Osmaniye/Osmaniye_20230208_10300500D9F8D400_post_disaster_2_0.json',
 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23/Osmaniye/Osmaniye_20221227_10300100DF069700_pre_disaster_7_9.json',
 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23/Osmaniye/Osmaniye_20221227_10300100DF069700_pre_disaster_5_11.json',
 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23/Osmaniye/Osmaniye_20221227_10300100DF069700_pre_disaster_12_4.json']

In [61]:
%%time
from tqdm.notebook import tqdm
## RUN
masks_dir = 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_mask_02-23'
label_dirs = 'turkey_earthquake/labels/copernicus_earthquake_turkey_tile_wkt_02-23'
label_files = list_json_files_recursive(label_dirs)

#ensure there is a directory
makedirs(masks_dir, exist_ok=True)
city_list = list_subdirectories(label_dirs)
# city dir
for city in city_list:
    # Ensure the directories exist
    Path(masks_dir+city+'/').mkdir(exist_ok=True)


#ensure there is a directory
makedirs(masks_dir, exist_ok=True)
for file_path in tqdm(sorted(label_files)):
    process_image_updated(file_path)

  0%|          | 0/443 [00:00<?, ?it/s]

CPU times: user 19.9 s, sys: 119 ms, total: 20 s
Wall time: 20.3 s
